<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/8%20-%20Chatbot/dante_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generare testo con le Reti Ricorrenti (LSTM)
In questo notebook vederemo come è possibile utilizzare le reti neurali ricorrenti LSTM non solamente per classificare del testo ma anche per generarlo ! Quello che andremo a fare è cercare di generare del nuovo testo con lo stesso stile che ha utilizzato Dante Alighieri per scrivere la Divina Commedia.<br><br>
Cominciamo scaricando una copia gratuita in TXT della Divina Commedia, puoi otterla da [questo sito internet](https://www.liberliber.it/online/autori/autori-a/dante-alighieri/la-divina-commedia-edizione-petrocchi/), se utilizzi Google Colaboratory o hai wget installato esegui pure il comando qui sotto per scaricare il file.

In [1]:
!wget https://www.liberliber.it/mediateca/libri/a/alighieri/la_divina_commedia/txt/la_divin.zip

--2019-04-24 12:47:40--  https://www.liberliber.it/mediateca/libri/a/alighieri/la_divina_commedia/txt/la_divin.zip
Resolving www.liberliber.it (www.liberliber.it)... 93.186.244.67
Connecting to www.liberliber.it (www.liberliber.it)|93.186.244.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232691 (227K) [application/zip]
Saving to: ‘la_divin.zip’

la_divin.zip        100%[===================>] 227.24K   663KB/s    in 0.3s    

2019-04-24 12:47:41 (663 KB/s) - ‘la_divin.zip’ saved [232691/232691]



ed estrai lo zip.

In [2]:
!unzip la_divin.zip

Archive:  la_divin.zip
  inflating: la_divin.txt            


## Processiamo i dati
Apriamo il file appena scaricato, leggiamone il contenuto e stampiamo i primi 100 caratteri.

In [3]:
with open("la_divin.txt", encoding="latin-1") as divine_file:
  divine_txt = divine_file.read()
  
print(divine_txt[:100])

Dante Alighieri
La Divina Commedia

Questo e-book è stato realizzato anche grazie al sostegno di:
E-


Come vedi l'ebook contiene del testo che non ci interessa, usiamo il metodo *.find(text)* per trovare dove inizia e finisce la divina commedia ed eseguiamo lo slicing per tenere soltanto il testo scritto da Dante.

In [5]:
start = divine_txt.find("Nel mezzo del cammin di nostra vita")
end = divine_txt.find("l'amor che move il sole e l'altre stelle.")

divine_txt = divine_txt[start:end]

print(divine_txt[:100])

Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura,
ché la diritta via era smarrit


Ogni canto contiene una piccola introduzione, ad esempio il primo:
<br><br>
LA DIVINA COMMEDIA
<br>
di Dante Alighieri
<br>
<br>
INFERNO
<br>
<br>
CANTO I
<br>
[Incomincia la Comedia di Dante Alleghieri di Fiorenza, ne la quale tratta de le pene e punimenti de' vizi e de' meriti e premi de le virt˘. Comincia il canto primo de la prima parte la quale si chiama Inferno, nel qual l'auttore fa proemio a tutta l'opera.]<br>
<br><br>
Il pattern è uguale per ogni canto, quindi possiamo rimuoverlo con un po' di codice:
 - Usiamo un'espressione regolare per rimuovere tutte le parole che cominciano con almeno due lettere maiscuole.
 - Usiamo un'altra espressione regolare per rimuovere tutte le frasi contenute tra parentesi quadre.
 - Rimuoviamo ogni occorrenza della frase 'di Dante Alighieri' dal testo.
 

In [0]:
import re

divine_txt = re.sub("[\(\[].*?[\)\]]", "", divine_txt)
divine_txt = re.sub("[A-Z][A-Z]+","",divine_txt)

divine_txt = divine_txt.replace("di Dante Alighieri","")

Usiamo la solita espressione regolare per rimuovere la punteggiatura, poi rimuoviamo anche i caratteri di 'a capo' e convertiamo tutto il testo in minuscolo.

In [0]:
divine_txt = re.sub(r'[^\w\s]','',divine_txt)
divine_txt = divine_txt.replace("\n"," ")
divine_txt = divine_txt.lower()

Adesso siamo pronti per tokenizzare il testo, usiamo spacy per farlo. Se non lo abbiamo già fatto installiamo il modulo per la lingua italiana.

In [8]:
!python -m spacy download it_core_news_sm

    100% |████████████████████████████████| 36.5MB 32.0MB/s 
  Running setup.py install for it-core-news-sm ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/it_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/it_core_news_sm

    You can now load the model via spacy.load('it_core_news_sm')



Carichiamolo, definiamo una funzione che estrae i token da tutto il testo ed utilizziamola.

In [9]:
import spacy

nlp = spacy.load("it_core_news_sm")

def preprocess(text):

  tokens = nlp(text)
  tokens_filtered = [token.text for token in tokens]
  return tokens_filtered

tokens = preprocess(divine_txt)
tokens[:10]

['nel',
 'mezzo',
 'del',
 'cammin',
 'di',
 'nostra',
 'vita',
 'mi',
 'ritrovai',
 'per']

Suddividiamo il testo in sequenze con una lunghezza massima di 10 parole, lo scopo delle nostra rete sarà quello di predire l'ultima parola della sequenza utilizzando quelle precedenti, per quale motivo utilizziamo questo approccio ? Lo vedrremo più avanti, per adesso fidati.

In [10]:
maxlen = 10

divine_sents = []

for i in range(maxlen, len(tokens)):
  divine_sents.append(tokens[i-maxlen:i])
  
print(divine_sents[0])
print(divine_sents[1])

['nel', 'mezzo', 'del', 'cammin', 'di', 'nostra', 'vita', 'mi', 'ritrovai', 'per']
['mezzo', 'del', 'cammin', 'di', 'nostra', 'vita', 'mi', 'ritrovai', 'per', 'una']


Adesso dobbiamo codificare le parole in numeri, possiamo farlo creandoci un dizionario di tutte le parole contenute nel testo e poi sostituire i token di ogni frase con la corrispondente posizione della parola nel dizionario. Per farlo possiamo usare direttamente la classe *Tokenizer* di keras che fa tutto per noi.

In [11]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(divine_sents)
divine_sents = tokenizer.texts_to_sequences(divine_sents)

divine_sents[0]

Using TensorFlow backend.


[38, 224, 24, 603, 4, 186, 153, 15, 13574, 7]

Creiamo i set con features e target, come già detto i features saranno i tokens di una seguenza eccetto l'ultimo, il target sarà invece proprio quest'ultimo token.

In [0]:
import numpy as np
divine_sents = np.array(divine_sents)

X = divine_sents[:,:-1]
y = divine_sents[:,-1]

Il nostro si tratta di un problema di classificazione multiclasse, le cui possibili classi sono tutte le parole contenute nel dizionario, vediamo quante sono esattamente.

In [13]:
vocab_size = len(tokenizer.word_counts)
vocab_size

13574

Abbiamo in totale 13575 parole, usiamo la funzione *to_categorical(y)* di keras per eseguire il one hot encoding delle variabili target.

In [14]:
from keras.utils import to_categorical

y = to_categorical(y, num_classes=vocab_size+1)
y.shape

(97393, 13575)

## Creazione della Rete Ricorrente
Creiamo la nostra architettura di rete neurale ricorrente:
 - Utilizziamo il *Word Embedding* per creare una rappresentazione vettoriale delle parole, addestrandolo sul nostro corpus di testo.
 - Aggiungiamo due strati ricorrenti con 50 nodi ciascuno, il primo dei quali dovrà ritornare una sequenza che servirà come input per il secondo.
 - Aggiungiamo un terzo strato denso con sempre 50 nodi.
 - Infine inseriamo uno strato di output con un numero di nodi ovviamente pari al numero di parole nel dizionario.

In [0]:
from keras import Model, Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(vocab_size+1, maxlen-1, input_length=maxlen-1))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation="relu"))
model.add(Dense(vocab_size+1, activation="softmax"))

Compiliamo il modello, trattandosi di un problema di classificazione multiclasse useremo la *categorical crossentropy* come funzione di costo, come algoritmo di ottimizzazione usiamo *rmsprop* che dovrebbe portare a migliori risultati se utilizziamo le reti ricorrenti.

In [0]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Come abbiamo detto, quello che vogliamo fare è creare una rete neurale in grado di generare testo dantesco, quindi perché stiamo eseguendo una classificazione ? L'utilizzo che faremo della rete è il seguente:
1. Forniremo alla rete del testo 'seed' di una lunghezza prestabilita, cioè del testo di base che poi essa userà per generare quello seguente, possiamo definire noi tale testo oppure estrarlo a caso dal corpus.
2. La rete predirà la parola che secondo essa dovrebbe seguire il testo 'seed'.
3. Aggiungiamo la parola predetta al testo.
4. Rimuoviamo la prima parola del testo.
5. Ripetiamo i punti da 2 a 4 fino a quando il testo predetto non avrà la lunghezza che vogliamo.

Definiamo una funzione che fa esattamente questo.

In [0]:
from random import randint
from keras.preprocessing.sequence import pad_sequences

def generate(seed=None, rand_seed_len=10, generate_len=25):
  
  output = ""
  
  if(seed==None):
    start_index = randint(0, len(divine_txt))
    text = divine_txt[start_index:start_index+rand_seed_len]
  else:
    text=seed
    
  for i in range(generate_len):
    tokens = np.array(tokenizer.texts_to_sequences([text]))
    tokens = pad_sequences(tokens, maxlen=maxlen-1, truncating="pre")
      
    pred_word = model.predict_classes([tokens])[0]
    pred_word = tokenizer.index_word[pred_word]
      
    text+=" "+pred_word
    output+=pred_word+" "
    
  return output

Utilizzando keras è possibile definire una funzione che viene eseguita al termine di ogni epoca dell'addestramento, definiamo una funzione che chiama la funzione per generare il testo, in modo tale da vedere come la qualità del testo varia durante l'addestramento.

In [0]:
def generate_on_epoch(epoch, _):
  output = generate()
  print('Dante dice: "'+output+'"')

Adesso siamo pronti per l'addestramento, per chiamare la funzione appena definita ad ogni epoca dobbiamo utilizzare i callback.
Creiamo un Lambda Callback passando all'interno del parametro on_epoch_end il nome della funzione.
Aggiungiamo il callback all'interno del parametro *callbacks* del metodo *.fit()*.
<br>
Keras ci mette a disposizone diversi callbacks da utilizzare durante l'addestramento, un'altro molto utile è quello per eseguire **l'early stopping**, cioè quella tecnica che ci permette di terminare l'addestramento in anticipo se la qualità del modello non sta migliorando. Utilizziamo l'early stopping con la classe *EarlyStopping* utilizzando i parametri *min_delta* e *patience* per interrompere l'addestramento se il valore della log loss non migliora di almeno 0.001 dopo 5 epoche.
<br><br>
**NOTA BENE**
<br>
Se non hai una GPU che supporta la tecnologia CUDA e non vuoi usare Google Colaboratory, ti consiglio di importare il modello pre-addestrato eseguendo il codice che trovi nella cella poco più sotto, altrimenti l'addestramento potrebbe richiedere anche giorni e mettere sotto forte stress il tuo pc.

In [46]:
from keras.callbacks import EarlyStopping, LambdaCallback

epoch_end_callback = LambdaCallback(on_epoch_end=generate_on_epoch)
earlyStopping = EarlyStopping(min_delta=0.001, patience=5)
model.fit(X, y, batch_size=128, epochs=500, callbacks=[earlyStopping, epoch_end_callback])

Epoch 1/500
97393/97393 [==============================] - 36s 368us/step - loss: 7.4445 - acc: 0.0406


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Dante dice: "che che che che che che che che che che che che che che che che che che che che che che che che che "
Epoch 2/500
97393/97393 [==============================] - 33s 338us/step - loss: 7.0821 - acc: 0.0404
Dante dice: "e e e e e e e e e e e e e e e e e e e e e e e e e "
Epoch 3/500
97393/97393 [==============================] - 32s 325us/step - loss: 6.9634 - acc: 0.0462
Dante dice: "che che che che che che che che che che che che che che che che che che che che che che che che che "
Epoch 4/500
97393/97393 [==============================] - 33s 341us/step - loss: 6.8422 - acc: 0.0535
Dante dice: "e che che che che che che che che che che che che che che che che che che che che che che che che "
Epoch 5/500
97393/97393 [==============================] - 33s 342us/step - loss: 6.7532 - acc: 0.0568
Dante dice: "e l occhi e che la occhi e che che che che che che che che che che che che che che che che che "
Epoch 6/500
97393/97393 [==============================] - 33s 335us/s

Se stai usando una GPU che supporta la tecnologia CUDA, sul tuo computer o con Google Colaboratory, l'addestramento per 500 epoche dovrebbe richiedere un paio di ore, se non vuoi aspettare puoi ridurre il numero di epoche a non meno di 100 oppure importare il modello che ho già addestrato eseguendo il codice qui sotto.

In [0]:
from keras.models import load_model

model.save('dante2.h5')  # creates a HDF5 file 'my_model.h5'

In [49]:
model.evaluate(X,y)

97393/97393 [==============================] - 64s 657us/step


[2.1553208158663284, 0.5180454447481608]

In [0]:
from urllib.request import urlretrieve
from keras.models import load_model

model_file = "dante.h5"

# Scarichiamo il modello preaddestrato dalla repo github del corso
model_url = "https://github.com/ProfAI/nlp00/raw/master/9%20-%20Reti%20Ricorrenti%20e%20Text%20Generation/model/dante.h5"
urlretrieve(model_url, model_file)

# carichiamo il modello
model = load_model(model_file)

Proviamo a dialogare con il nostro Dante-bot, il testo che inseriremo verrà usato come seed per la generazione di nuovo testo in stile dantesco.

In [32]:
seed = ""

while(seed!="ciao"):
  seed = input("Io: ")
  generated = generate(seed=seed)
  print("Dante: "+generated)

Io: ciao
Dante: e la occhi e che e che e che e che e che e che e che e che e che e che e che 
